<a href="https://colab.research.google.com/github/Park-YounHo/pytoch/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import urllib.request
import zipfile
import tarfile
import glob
import io

In [2]:
data_dir = "./data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [3]:
vocab_dir = "./vocab/"
if not os.path.exists(vocab_dir):
    os.mkdir(vocab_dir)

In [4]:
weights_dir = "./weights/"
if not os.path.exists(weights_dir):
    os.mkdir(weights_dir)

In [5]:
save_path="./vocab/bert-base-uncased-vocab.txt"
url = "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt"
urllib.request.urlretrieve(url, save_path)

('./vocab/bert-base-uncased-vocab.txt',
 <http.client.HTTPMessage at 0x7fa21fc143d0>)

In [6]:
save_path = "./weights/bert-base-uncased.tar.gz"
url = "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz"
urllib.request.urlretrieve(url, save_path)

archive_file = "./weights/bert-base-uncased.tar.gz" 
tar = tarfile.open(archive_file, 'r:gz')
tar.extractall('./weights/')
tar.close()

In [7]:
target_dir_path="./data/"

if not os.path.exists(target_dir_path):
    os.mkdir(target_dir_path)
    
url = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
save_path = "./data/aclImdb_v1.tar.gz"
urllib.request.urlretrieve(url, save_path)

tar = tarfile.open('./data/aclImdb_v1.tar.gz')
tar.extractall('./data/')
tar.close()

In [8]:
target_dir_path="./data/aclImdb/"

if os.path.exists(target_dir_path):
  
    f=open('./data/IMDb_train.tsv','w')

    path = './data/aclImdb/train/pos/'
    for fname in glob.glob(os.path.join(path,'*.txt')):
        with io.open(fname, 'r', encoding="utf-8") as ff:
            text = ff.readline()
            
            text = text.replace('\t', " ")
            
            text = text+'\t'+'1'+'\t'+'\n'
            f.write(text)

    path = './data/aclImdb/train/neg/'
    for fname in glob.glob(os.path.join(path,'*.txt')):
        with io.open(fname, 'r', encoding="utf-8") as ff:
            text = ff.readline()
            
            text = text.replace('\t', " ")
            
            text = text+'\t'+'0'+'\t'+'\n'
            f.write(text)

    f.close()
    
    
    f=open('./data/IMDb_test.tsv','w')

    path = './data/aclImdb/test/pos/'
    for fname in glob.glob(os.path.join(path,'*.txt')):
        with io.open(fname, 'r', encoding="utf-8") as ff:
            text = ff.readline()
            
            text = text.replace('\t', " ")
        
            text = text+'\t'+'1'+'\t'+'\n'
            f.write(text)

    path = './data/aclImdb/test/neg/'

    for fname in glob.glob(os.path.join(path,'*.txt')):
        with io.open(fname, 'r', encoding="utf-8") as ff:
            text = ff.readline()
            
            
            text = text.replace('\t', " ")
            
            text = text+'\t'+'0'+'\t'+'\n'
            f.write(text)


    f.close()

In [9]:
import math
import numpy as np

import torch
from torch import nn

In [10]:
import json

config_file = "./weights/bert_config.json"

json_file = open(config_file, 'r')
config = json.load(json_file)

config

{'attention_probs_dropout_prob': 0.1,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'max_position_embeddings': 512,
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'type_vocab_size': 2,
 'vocab_size': 30522}

In [11]:
config['hidden_size']

768

In [12]:
pip install attrdict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
from attrdict import AttrDict

config = AttrDict(config)
config.hidden_size

768

In [14]:
class BertLayerNorm(nn.Module):

    def __init__(self, hidden_size, eps=1e-12):
        super(BertLayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(hidden_size))
        self.beta = nn.Parameter(torch.zeros(hidden_size))
        self.variance_epsilon = eps

    def forward(self, x):
        u = x.mean(-1, keepdim=True)
        s = (x - u).pow(2).mean(-1, keepdim=True)
        x = (x - u) / torch.sqrt(s + self.variance_epsilon)
        return self.gamma * x + self.beta

In [15]:
class BertEmbeddings(nn.Module):

    def __init__(self, config):
        super(BertEmbeddings, self).__init__()


        self.word_embeddings = nn.Embedding(
            config.vocab_size, config.hidden_size, padding_idx=0)

        self.position_embeddings = nn.Embedding(
            config.max_position_embeddings, config.hidden_size)

        self.token_type_embeddings = nn.Embedding(
            config.type_vocab_size, config.hidden_size)

        self.LayerNorm = BertLayerNorm(config.hidden_size, eps=1e-12)

        # Dropout　'hidden_dropout_prob': 0.1
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, input_ids, token_type_ids=None):

        words_embeddings = self.word_embeddings(input_ids)

        if token_type_ids is None:
            token_type_ids = torch.zeros_like(input_ids)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)

        seq_length = input_ids.size(1)
        position_ids = torch.arange(
            seq_length, dtype=torch.long, device=input_ids.device)
        position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
        position_embeddings = self.position_embeddings(position_ids)

        embeddings = words_embeddings + position_embeddings + token_type_embeddings

        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)

        return embeddings

In [16]:
class BertLayer(nn.Module):

    def __init__(self, config):
        super(BertLayer, self).__init__()

        self.attention = BertAttention(config)

        self.intermediate = BertIntermediate(config)

        self.output = BertOutput(config)

    def forward(self, hidden_states, attention_mask, attention_show_flg=False):

        if attention_show_flg == True:
            attention_output, attention_probs = self.attention(
                hidden_states, attention_mask, attention_show_flg)
            intermediate_output = self.intermediate(attention_output)
            layer_output = self.output(intermediate_output, attention_output)
            return layer_output, attention_probs

        elif attention_show_flg == False:
            attention_output = self.attention(
                hidden_states, attention_mask, attention_show_flg)
            intermediate_output = self.intermediate(attention_output)
            layer_output = self.output(intermediate_output, attention_output)

            return layer_output

In [17]:
class BertAttention(nn.Module):
    def __init__(self, config):
        super(BertAttention, self).__init__()
        self.selfattn = BertSelfAttention(config)
        self.output = BertSelfOutput(config)

    def forward(self, input_tensor, attention_mask, attention_show_flg=False):
        if attention_show_flg == True:
            self_output, attention_probs = self.selfattn(input_tensor, attention_mask, attention_show_flg)
            attention_output = self.output(self_output, input_tensor)
            return attention_output, attention_probs
        
        elif attention_show_flg == False:
            self_output = self.selfattn(input_tensor, attention_mask, attention_show_flg)
            attention_output = self.output(self_output, input_tensor)
            return attention_output

In [18]:
class BertSelfAttention(nn.Module):

    def __init__(self, config):
        super(BertSelfAttention, self).__init__()

        self.num_attention_heads = config.num_attention_heads

        self.attention_head_size = int(
            config.hidden_size / config.num_attention_heads)
        self.all_head_size = self.num_attention_heads * \
            self.attention_head_size

        self.query = nn.Linear(config.hidden_size, self.all_head_size)
        self.key = nn.Linear(config.hidden_size, self.all_head_size)
        self.value = nn.Linear(config.hidden_size, self.all_head_size)

        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)

    def transpose_for_scores(self, x):
        new_x_shape = x.size()[
            :-1] + (self.num_attention_heads, self.attention_head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)

    def forward(self, hidden_states, attention_mask, attention_show_flg=False):

        mixed_query_layer = self.query(hidden_states)
        mixed_key_layer = self.key(hidden_states)
        mixed_value_layer = self.value(hidden_states)

        query_layer = self.transpose_for_scores(mixed_query_layer)
        key_layer = self.transpose_for_scores(mixed_key_layer)
        value_layer = self.transpose_for_scores(mixed_value_layer)

        attention_scores = torch.matmul(
            query_layer, key_layer.transpose(-1, -2))
        attention_scores = attention_scores / \
            math.sqrt(self.attention_head_size)

        attention_scores = attention_scores + attention_mask

        attention_probs = nn.Softmax(dim=-1)(attention_scores)

        attention_probs = self.dropout(attention_probs)

        context_layer = torch.matmul(attention_probs, value_layer)

        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[
            :-2] + (self.all_head_size,)
        context_layer = context_layer.view(*new_context_layer_shape)

        if attention_show_flg == True:
            return context_layer, attention_probs
        elif attention_show_flg == False:
            return context_layer

In [19]:
class BertSelfOutput(nn.Module):

    def __init__(self, config):
        super(BertSelfOutput, self).__init__()

        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.LayerNorm = BertLayerNorm(config.hidden_size, eps=1e-12)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, hidden_states, input_tensor):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states

In [20]:
def gelu(x):
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

class BertIntermediate(nn.Module):
    def __init__(self, config):
        super(BertIntermediate, self).__init__()
        
        self.dense = nn.Linear(config.hidden_size, config.intermediate_size)
        
        self.intermediate_act_fn = gelu
            
    def forward(self, hidden_states):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.intermediate_act_fn(hidden_states)
        return hidden_states

In [21]:
class BertOutput(nn.Module):

    def __init__(self, config):
        super(BertOutput, self).__init__()

        self.dense = nn.Linear(config.intermediate_size, config.hidden_size)

        self.LayerNorm = BertLayerNorm(config.hidden_size, eps=1e-12)

        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, hidden_states, input_tensor):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states

In [22]:
class BertEncoder(nn.Module):
    def __init__(self, config):
        super(BertEncoder, self).__init__()

        self.layer = nn.ModuleList([BertLayer(config)
                                    for _ in range(config.num_hidden_layers)])

    def forward(self, hidden_states, attention_mask, output_all_encoded_layers=True, attention_show_flg=False):

        all_encoder_layers = []

        for layer_module in self.layer:

            if attention_show_flg == True:
                hidden_states, attention_probs = layer_module(
                    hidden_states, attention_mask, attention_show_flg)
            elif attention_show_flg == False:
                hidden_states = layer_module(
                    hidden_states, attention_mask, attention_show_flg)

            if output_all_encoded_layers:
                all_encoder_layers.append(hidden_states)

        if not output_all_encoded_layers:
            all_encoder_layers.append(hidden_states)

        if attention_show_flg == True:
            return all_encoder_layers, attention_probs
        elif attention_show_flg == False:
            return all_encoder_layers

In [23]:
class BertPooler(nn.Module):

    def __init__(self, config):
        super(BertPooler, self).__init__()

        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()

    def forward(self, hidden_states):
        first_token_tensor = hidden_states[:, 0]

        pooled_output = self.dense(first_token_tensor)

        pooled_output = self.activation(pooled_output)

        return pooled_output

In [24]:
input_ids = torch.LongTensor([[31, 51, 12, 23, 99], [15, 5, 1, 0, 0]])
print("입력 단어 ID열의 텐서 크기: ", input_ids.shape)

attention_mask = torch.LongTensor([[1, 1, 1, 1, 1], [1, 1, 1, 0, 0]])
print("입력 마스크의 텐서 크기: ", attention_mask.shape)

token_type_ids = torch.LongTensor([[0, 0, 1, 1, 1], [0, 1, 1, 1, 1]])
print("입력 문장 ID의 텐서 크기: ", token_type_ids.shape)


embeddings = BertEmbeddings(config)
encoder = BertEncoder(config)
pooler = BertPooler(config)

extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
extended_attention_mask = extended_attention_mask.to(dtype=torch.float32)
extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0
print("확장된 마스크의 텐서 크기: ", extended_attention_mask.shape)

out1 = embeddings(input_ids, token_type_ids)
print("BertEmbeddings의 출력 텐서 크기: ", out1.shape)

out2 = encoder(out1, extended_attention_mask)
print("BertEncoder 최후 층의 출력 텐서 크기: ", out2[0].shape)

out3 = pooler(out2[-1])
print("BertPooler의 출력 텐서 크기: ", out3.shape)

입력 단어 ID열의 텐서 크기:  torch.Size([2, 5])
입력 마스크의 텐서 크기:  torch.Size([2, 5])
입력 문장 ID의 텐서 크기:  torch.Size([2, 5])
확장된 마스크의 텐서 크기:  torch.Size([2, 1, 1, 5])
BertEmbeddings의 출력 텐서 크기:  torch.Size([2, 5, 768])
BertEncoder 최후 층의 출력 텐서 크기:  torch.Size([2, 5, 768])
BertPooler의 출력 텐서 크기:  torch.Size([2, 768])


In [25]:
class BertModel(nn.Module):

    def __init__(self, config):
        super(BertModel, self).__init__()

        self.embeddings = BertEmbeddings(config)
        self.encoder = BertEncoder(config)
        self.pooler = BertPooler(config)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, output_all_encoded_layers=True, attention_show_flg=False):

        if attention_mask is None:
            attention_mask = torch.ones_like(input_ids)
        if token_type_ids is None:
            token_type_ids = torch.zeros_like(input_ids)

        extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)

        extended_attention_mask = extended_attention_mask.to(
            dtype=torch.float32)
        extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0

        embedding_output = self.embeddings(input_ids, token_type_ids)

        if attention_show_flg == True:

            encoded_layers, attention_probs = self.encoder(embedding_output,
                                                           extended_attention_mask,
                                                           output_all_encoded_layers, attention_show_flg)

        elif attention_show_flg == False:
            encoded_layers = self.encoder(embedding_output,
                                          extended_attention_mask,
                                          output_all_encoded_layers, attention_show_flg)

        pooled_output = self.pooler(encoded_layers[-1])

        if not output_all_encoded_layers:
            encoded_layers = encoded_layers[-1]

        if attention_show_flg == True:
            return encoded_layers, pooled_output, attention_probs
        elif attention_show_flg == False:
            return encoded_layers, pooled_output

In [26]:
input_ids = torch.LongTensor([[31, 51, 12, 23, 99], [15, 5, 1, 0, 0]])
attention_mask = torch.LongTensor([[1, 1, 1, 1, 1], [1, 1, 1, 0, 0]])
token_type_ids = torch.LongTensor([[0, 0, 1, 1, 1], [0, 1, 1, 1, 1]])

net = BertModel(config)

encoded_layers, pooled_output, attention_probs = net(
    input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False, attention_show_flg=True)

print("encoded_layers의 텐서 크기: ", encoded_layers.shape)
print("pooled_output의 텐서 크기: ", pooled_output.shape)
print("attention_probs의 텐서 크기: ", attention_probs.shape)

encoded_layers의 텐서 크기:  torch.Size([2, 5, 768])
pooled_output의 텐서 크기:  torch.Size([2, 768])
attention_probs의 텐서 크기:  torch.Size([2, 12, 5, 5])


In [27]:
weights_path = "./weights/pytorch_model.bin"
loaded_state_dict = torch.load(weights_path)

for s in loaded_state_dict.keys():
    print(s)

bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.gamma
bert.embeddings.LayerNorm.beta
bert.encoder.layer.0.attention.self.query.weight
bert.encoder.layer.0.attention.self.query.bias
bert.encoder.layer.0.attention.self.key.weight
bert.encoder.layer.0.attention.self.key.bias
bert.encoder.layer.0.attention.self.value.weight
bert.encoder.layer.0.attention.self.value.bias
bert.encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.bias
bert.encoder.layer.0.attention.output.LayerNorm.gamma
bert.encoder.layer.0.attention.output.LayerNorm.beta
bert.encoder.layer.0.intermediate.dense.weight
bert.encoder.layer.0.intermediate.dense.bias
bert.encoder.layer.0.output.dense.weight
bert.encoder.layer.0.output.dense.bias
bert.encoder.layer.0.output.LayerNorm.gamma
bert.encoder.layer.0.output.LayerNorm.beta
bert.encoder.layer.1.attention.self.query.weight
bert.encode

In [28]:
net = BertModel(config)
net.eval()

param_names = []

for name, param in net.named_parameters():
    print(name)
    param_names.append(name)

embeddings.word_embeddings.weight
embeddings.position_embeddings.weight
embeddings.token_type_embeddings.weight
embeddings.LayerNorm.gamma
embeddings.LayerNorm.beta
encoder.layer.0.attention.selfattn.query.weight
encoder.layer.0.attention.selfattn.query.bias
encoder.layer.0.attention.selfattn.key.weight
encoder.layer.0.attention.selfattn.key.bias
encoder.layer.0.attention.selfattn.value.weight
encoder.layer.0.attention.selfattn.value.bias
encoder.layer.0.attention.output.dense.weight
encoder.layer.0.attention.output.dense.bias
encoder.layer.0.attention.output.LayerNorm.gamma
encoder.layer.0.attention.output.LayerNorm.beta
encoder.layer.0.intermediate.dense.weight
encoder.layer.0.intermediate.dense.bias
encoder.layer.0.output.dense.weight
encoder.layer.0.output.dense.bias
encoder.layer.0.output.LayerNorm.gamma
encoder.layer.0.output.LayerNorm.beta
encoder.layer.1.attention.selfattn.query.weight
encoder.layer.1.attention.selfattn.query.bias
encoder.layer.1.attention.selfattn.key.weight
e

In [29]:
new_state_dict = net.state_dict().copy()

for index, (key_name, value) in enumerate(loaded_state_dict.items()):
    name = param_names[index]  
    new_state_dict[name] = value 
    print(str(key_name)+"→"+str(name))

    if index+1 >= len(param_names):
        break

net.load_state_dict(new_state_dict)

bert.embeddings.word_embeddings.weight→embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight→embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight→embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.gamma→embeddings.LayerNorm.gamma
bert.embeddings.LayerNorm.beta→embeddings.LayerNorm.beta
bert.encoder.layer.0.attention.self.query.weight→encoder.layer.0.attention.selfattn.query.weight
bert.encoder.layer.0.attention.self.query.bias→encoder.layer.0.attention.selfattn.query.bias
bert.encoder.layer.0.attention.self.key.weight→encoder.layer.0.attention.selfattn.key.weight
bert.encoder.layer.0.attention.self.key.bias→encoder.layer.0.attention.selfattn.key.bias
bert.encoder.layer.0.attention.self.value.weight→encoder.layer.0.attention.selfattn.value.weight
bert.encoder.layer.0.attention.self.value.bias→encoder.layer.0.attention.selfattn.value.bias
bert.encoder.layer.0.attention.output.dense.weight→encoder.layer.0.attention.output

<All keys matched successfully>

In [30]:
import collections

def load_vocab(vocab_file):
    vocab = collections.OrderedDict()
    ids_to_tokens = collections.OrderedDict()
    index = 0

    with open(vocab_file, "r", encoding="utf-8") as reader:
        while True:
            token = reader.readline()
            if not token:
                break
            token = token.strip()

            vocab[token] = index
            ids_to_tokens[index] = token
            index += 1

    return vocab, ids_to_tokens


vocab_file = "./vocab/bert-base-uncased-vocab.txt"
vocab, ids_to_tokens = load_vocab(vocab_file)

In [31]:
vocab

OrderedDict([('[PAD]', 0),
             ('[unused0]', 1),
             ('[unused1]', 2),
             ('[unused2]', 3),
             ('[unused3]', 4),
             ('[unused4]', 5),
             ('[unused5]', 6),
             ('[unused6]', 7),
             ('[unused7]', 8),
             ('[unused8]', 9),
             ('[unused9]', 10),
             ('[unused10]', 11),
             ('[unused11]', 12),
             ('[unused12]', 13),
             ('[unused13]', 14),
             ('[unused14]', 15),
             ('[unused15]', 16),
             ('[unused16]', 17),
             ('[unused17]', 18),
             ('[unused18]', 19),
             ('[unused19]', 20),
             ('[unused20]', 21),
             ('[unused21]', 22),
             ('[unused22]', 23),
             ('[unused23]', 24),
             ('[unused24]', 25),
             ('[unused25]', 26),
             ('[unused26]', 27),
             ('[unused27]', 28),
             ('[unused28]', 29),
             ('[unused29]', 30),
  

In [32]:
ids_to_tokens

OrderedDict([(0, '[PAD]'),
             (1, '[unused0]'),
             (2, '[unused1]'),
             (3, '[unused2]'),
             (4, '[unused3]'),
             (5, '[unused4]'),
             (6, '[unused5]'),
             (7, '[unused6]'),
             (8, '[unused7]'),
             (9, '[unused8]'),
             (10, '[unused9]'),
             (11, '[unused10]'),
             (12, '[unused11]'),
             (13, '[unused12]'),
             (14, '[unused13]'),
             (15, '[unused14]'),
             (16, '[unused15]'),
             (17, '[unused16]'),
             (18, '[unused17]'),
             (19, '[unused18]'),
             (20, '[unused19]'),
             (21, '[unused20]'),
             (22, '[unused21]'),
             (23, '[unused22]'),
             (24, '[unused23]'),
             (25, '[unused24]'),
             (26, '[unused25]'),
             (27, '[unused26]'),
             (28, '[unused27]'),
             (29, '[unused28]'),
             (30, '[unused29]'),
  

In [33]:
from utils.tokenizer import BasicTokenizer, WordpieceTokenizer

class BertTokenizer(object):

    def __init__(self, vocab_file, do_lower_case=True):

        self.vocab, self.ids_to_tokens = load_vocab(vocab_file)

        never_split = ("[UNK]", "[SEP]", "[PAD]", "[CLS]", "[MASK]")

        self.basic_tokenizer = BasicTokenizer(do_lower_case=do_lower_case,
                                              never_split=never_split)
        self.wordpiece_tokenizer = WordpieceTokenizer(vocab=self.vocab)

    def tokenize(self, text):
        split_tokens = []
        for token in self.basic_tokenizer.tokenize(text):
            for sub_token in self.wordpiece_tokenizer.tokenize(token):
                split_tokens.append(sub_token)
        return split_tokens

    def convert_tokens_to_ids(self, tokens):
        ids = []
        for token in tokens:
            ids.append(self.vocab[token])

        return ids

    def convert_ids_to_tokens(self, ids):
        tokens = []
        for i in ids:
            tokens.append(self.ids_to_tokens[i])
        return tokens

In [34]:
text_1 = "[CLS] I accessed the bank account. [SEP]"

text_2 = "[CLS] He transferred the deposit money into the bank account. [SEP]"

text_3 = "[CLS] We play soccer at the bank of the river. [SEP]"

tokenizer = BertTokenizer(
    vocab_file="./vocab/bert-base-uncased-vocab.txt", do_lower_case=True)

tokenized_text_1 = tokenizer.tokenize(text_1)
tokenized_text_2 = tokenizer.tokenize(text_2)
tokenized_text_3 = tokenizer.tokenize(text_3)

print(tokenized_text_1)

['[CLS]', 'i', 'accessed', 'the', 'bank', 'account', '.', '[SEP]']


In [35]:
indexed_tokens_1 = tokenizer.convert_tokens_to_ids(tokenized_text_1)
indexed_tokens_2 = tokenizer.convert_tokens_to_ids(tokenized_text_2)
indexed_tokens_3 = tokenizer.convert_tokens_to_ids(tokenized_text_3)

bank_posi_1 = np.where(np.array(tokenized_text_1) == "bank")[0][0]  # 4
bank_posi_2 = np.where(np.array(tokenized_text_2) == "bank")[0][0]  # 8
bank_posi_3 = np.where(np.array(tokenized_text_3) == "bank")[0][0]  # 6

tokens_tensor_1 = torch.tensor([indexed_tokens_1])
tokens_tensor_2 = torch.tensor([indexed_tokens_2])
tokens_tensor_3 = torch.tensor([indexed_tokens_3])

bank_word_id = tokenizer.convert_tokens_to_ids(["bank"])[0]

print(tokens_tensor_1)

tensor([[  101,  1045, 11570,  1996,  2924,  4070,  1012,   102]])


In [36]:
with torch.no_grad():
    encoded_layers_1, _ = net(tokens_tensor_1, output_all_encoded_layers=True)
    encoded_layers_2, _ = net(tokens_tensor_2, output_all_encoded_layers=True)
    encoded_layers_3, _ = net(tokens_tensor_3, output_all_encoded_layers=True)

In [37]:
bank_vector_0 = net.embeddings.word_embeddings.weight[bank_word_id]

bank_vector_1_1 = encoded_layers_1[0][0, bank_posi_1]

bank_vector_1_12 = encoded_layers_1[11][0, bank_posi_1]

bank_vector_2_1 = encoded_layers_2[0][0, bank_posi_2]
bank_vector_2_12 = encoded_layers_2[11][0, bank_posi_2]
bank_vector_3_1 = encoded_layers_3[0][0, bank_posi_3]
bank_vector_3_12 = encoded_layers_3[11][0, bank_posi_3]

In [38]:
import torch.nn.functional as F

print("bank의 초기 벡터와 문장1의 1단 bank의 유사도: ",
      F.cosine_similarity(bank_vector_0, bank_vector_1_1, dim=0))
print("bank의 초기 벡터와 문장1의 12단 bank의 유사도: ",
      F.cosine_similarity(bank_vector_0, bank_vector_1_12, dim=0))

print("문장1의 1층 bank와 문장2의 1단 bank의 유사도: ",
      F.cosine_similarity(bank_vector_1_1, bank_vector_2_1, dim=0))
print("문장1의 1층 bank와 문장3의 1단 bank의 유사도: ",
      F.cosine_similarity(bank_vector_1_1, bank_vector_3_1, dim=0))

print("문장1의 12층 bank와 문장2의 12단 bank의 유사도: ",
      F.cosine_similarity(bank_vector_1_12, bank_vector_2_12, dim=0))
print("문장1의 12층 bank와 문장3의 12단 bank의 유사도: ",
      F.cosine_similarity(bank_vector_1_12, bank_vector_3_12, dim=0))

bank의 초기 벡터와 문장1의 1단 bank의 유사도:  tensor(0.6814, grad_fn=<DivBackward0>)
bank의 초기 벡터와 문장1의 12단 bank의 유사도:  tensor(0.2276, grad_fn=<DivBackward0>)
문장1의 1층 bank와 문장2의 1단 bank의 유사도:  tensor(0.8968)
문장1의 1층 bank와 문장3의 1단 bank의 유사도:  tensor(0.7584)
문장1의 12층 bank와 문장2의 12단 bank의 유사도:  tensor(0.8796)
문장1의 12층 bank와 문장3의 12단 bank의 유사도:  tensor(0.4814)


In [39]:
pip install torchtext==0.8.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
import random
import time
import numpy as np
from tqdm import tqdm
import torch 
from torch import nn
import torch.optim as optim
import torchtext

In [41]:
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [42]:
import re
import string
from utils.bert import BertTokenizer


def preprocessing_text(text):
    text = re.sub('<br />', '', text)

    for p in string.punctuation:
        if (p == ".") or (p == ","):
            continue
        else:
            text = text.replace(p, " ")

    text = text.replace(".", " . ")
    text = text.replace(",", " , ")
    return text


tokenizer_bert = BertTokenizer(
    vocab_file="./vocab/bert-base-uncased-vocab.txt", do_lower_case=True)


def tokenizer_with_preprocessing(text, tokenizer=tokenizer_bert.tokenize):
    text = preprocessing_text(text)
    ret = tokenizer(text)
    return ret

In [43]:
max_length = 256

TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, use_vocab=True,
                            lower=True, include_lengths=True, batch_first=True, fix_length=max_length, init_token="[CLS]", eos_token="[SEP]", pad_token='[PAD]', unk_token='[UNK]')
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [44]:
train_val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='IMDb_train.tsv',
    test='IMDb_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

train_ds, val_ds = train_val_ds.split(
    split_ratio=0.8, random_state=random.seed(1234))

/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [45]:
from utils.bert import BertTokenizer, load_vocab

vocab_bert, ids_to_tokens_bert = load_vocab(
    vocab_file="./vocab/bert-base-uncased-vocab.txt")


TEXT.build_vocab(train_ds, min_freq=1)
TEXT.vocab.stoi = vocab_bert

In [46]:
batch_size = 32

train_dl = torchtext.data.Iterator(
    train_ds, batch_size=batch_size, train=True)

val_dl = torchtext.data.Iterator(
    val_ds, batch_size=batch_size, train=False, sort=False)

test_dl = torchtext.data.Iterator(
    test_ds, batch_size=batch_size, train=False, sort=False)

dataloaders_dict = {"train": train_dl, "val": val_dl}

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: Iterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [47]:
batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)


(tensor([[ 101, 2061, 2411,  ...,    0,    0,    0],
        [ 101, 2043, 1045,  ...,    0,    0,    0],
        [ 101, 2023, 3185,  ...,    0,    0,    0],
        ...,
        [ 101, 6203, 2643,  ...,    0,    0,    0],
        [ 101, 2023, 2003,  ...,    0,    0,    0],
        [ 101, 2116, 2111,  ..., 3496, 2003,  102]]), tensor([147,  77, 175, 138,  49, 232, 231, 142, 165, 232,  17, 232, 161, 230,
        175, 256, 152, 256, 204, 256, 129, 221, 166, 149, 211, 256, 201, 161,
        256, 148, 196, 256]))
tensor([1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0,
        0, 1, 1, 1, 0, 1, 1, 1])


/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [48]:
text_minibatch_1 = (batch.Text[0][1]).numpy()

text = tokenizer_bert.convert_ids_to_tokens(text_minibatch_1)

print(text)

['[CLS]', 'when', 'i', 'learned', 'of', 'sir', 'alec', 'guinness', 'death', ',', 'this', 'was', 'the', 'first', 'of', 'his', 'many', 'films', 'i', 'thought', 'of', 're', 'seeing', '.', 'what', 'a', 'wonderful', 'dr', '##oll', 'commentary', 'the', 'film', 'provides', 'even', 'after', 'all', 'these', 'years', '.', 'and', 'guinness', 'helps', 'to', 'weave', 'the', 'charm', 'into', 'every', 'frame', '.', 'his', 'eyes', 'and', 'face', 'are', 'as', 'luminous', 'as', 'that', 'white', 'suit', 'he', 'wears', '.', 'both', 'he', 'and', 'the', 'film', 'have', 'to', 'be', 'considered', 'lifetime', 'favorites', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[P

In [49]:
from utils.bert import get_config, BertModel, set_learned_params

config = get_config(file_path="./weights/bert_config.json")

net_bert = BertModel(config)

net_bert = set_learned_params(
    net_bert, weights_path="./weights/pytorch_model.bin")

bert.embeddings.word_embeddings.weight→embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight→embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight→embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.gamma→embeddings.LayerNorm.gamma
bert.embeddings.LayerNorm.beta→embeddings.LayerNorm.beta
bert.encoder.layer.0.attention.self.query.weight→encoder.layer.0.attention.selfattn.query.weight
bert.encoder.layer.0.attention.self.query.bias→encoder.layer.0.attention.selfattn.query.bias
bert.encoder.layer.0.attention.self.key.weight→encoder.layer.0.attention.selfattn.key.weight
bert.encoder.layer.0.attention.self.key.bias→encoder.layer.0.attention.selfattn.key.bias
bert.encoder.layer.0.attention.self.value.weight→encoder.layer.0.attention.selfattn.value.weight
bert.encoder.layer.0.attention.self.value.bias→encoder.layer.0.attention.selfattn.value.bias
bert.encoder.layer.0.attention.output.dense.weight→encoder.layer.0.attention.output

In [50]:
class BertForIMDb(nn.Module):

    def __init__(self, net_bert):
        super(BertForIMDb, self).__init__()

        self.bert = net_bert

        self.cls = nn.Linear(in_features=768, out_features=2)

        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, output_all_encoded_layers=False, attention_show_flg=False):

        if attention_show_flg == True:
            encoded_layers, pooled_output, attention_probs = self.bert(
                input_ids, token_type_ids, attention_mask, output_all_encoded_layers, attention_show_flg)
        elif attention_show_flg == False:
            encoded_layers, pooled_output = self.bert(
                input_ids, token_type_ids, attention_mask, output_all_encoded_layers, attention_show_flg)

        vec_0 = encoded_layers[:, 0, :]
        vec_0 = vec_0.view(-1, 768)
        out = self.cls(vec_0)

        if attention_show_flg == True:
            return out, attention_probs
        elif attention_show_flg == False:
            return out

In [51]:
net = BertForIMDb(net_bert)

net.train()

print('네트워크 설정 완료')

네트워크 설정 완료


In [52]:
for name, param in net.named_parameters():
    param.requires_grad = False

for name, param in net.bert.encoder.layer[-1].named_parameters():
    param.requires_grad = True

for name, param in net.cls.named_parameters():
    param.requires_grad = True

In [53]:
optimizer = optim.Adam([
    {'params': net.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
    {'params': net.cls.parameters(), 'lr': 5e-5}
], betas=(0.9, 0.999))

criterion = nn.CrossEntropyLoss()

In [54]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("사용 장치: ", device)
    print('-----start-------')

    net.to(device)

    torch.backends.cudnn.benchmark = True

    batch_size = dataloaders_dict["train"].batch_size

    for epoch in range(num_epochs):
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()
            else:
                net.eval()

            epoch_loss = 0.0 
            epoch_corrects = 0  
            iteration = 1

            t_epoch_start = time.time()
            t_iter_start = time.time()

            for batch in (dataloaders_dict[phase]):

                inputs = batch.Text[0].to(device)  
                labels = batch.Label.to(device)  

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):

                    outputs = net(inputs, token_type_ids=None, attention_mask=None,
                                  output_all_encoded_layers=False, attention_show_flg=False)

                    loss = criterion(outputs, labels)  

                    _, preds = torch.max(outputs, 1)  

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                        if (iteration % 10 == 0): 
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            acc = (torch.sum(preds == labels.data)
                                   ).double()/batch_size
                            print('반복 {} || Loss: {:.4f} || 10iter: {:.4f} sec. || 이 반복의 정답률: {}'.format(
                                iteration, loss.item(), duration, acc))
                            t_iter_start = time.time()

                    iteration += 1

                    epoch_loss += loss.item() * batch_size
                    epoch_corrects += torch.sum(preds == labels.data)

            t_epoch_finish = time.time()
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))
            t_epoch_start = time.time()

    return net

In [55]:
num_epochs = 2
net_trained = train_model(net, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)


사용 장치:  cuda:0
-----start-------
반복 10 || Loss: 0.7150 || 10iter: 5.4590 sec. || 이 반복의 정답률: 0.46875
반복 20 || Loss: 0.6284 || 10iter: 5.2917 sec. || 이 반복의 정답률: 0.625
반복 30 || Loss: 0.6160 || 10iter: 5.3253 sec. || 이 반복의 정답률: 0.78125
반복 40 || Loss: 0.6208 || 10iter: 5.3477 sec. || 이 반복의 정답률: 0.71875
반복 50 || Loss: 0.5847 || 10iter: 5.4007 sec. || 이 반복의 정답률: 0.6875
반복 60 || Loss: 0.5435 || 10iter: 5.4160 sec. || 이 반복의 정답률: 0.8125
반복 70 || Loss: 0.5384 || 10iter: 5.4475 sec. || 이 반복의 정답률: 0.75
반복 80 || Loss: 0.5156 || 10iter: 5.4738 sec. || 이 반복의 정답률: 0.6875
반복 90 || Loss: 0.3153 || 10iter: 5.4967 sec. || 이 반복의 정답률: 0.875
반복 100 || Loss: 0.3462 || 10iter: 5.5243 sec. || 이 반복의 정답률: 0.875
반복 110 || Loss: 0.3460 || 10iter: 5.5479 sec. || 이 반복의 정답률: 0.875
반복 120 || Loss: 0.3068 || 10iter: 5.5873 sec. || 이 반복의 정답률: 0.8125
반복 130 || Loss: 0.3210 || 10iter: 5.6514 sec. || 이 반복의 정답률: 0.84375
반복 140 || Loss: 0.4766 || 10iter: 5.6657 sec. || 이 반복의 정답률: 0.875
반복 150 || Loss: 0.4593 || 10iter: 5.7083 

In [56]:
save_path = './weights/bert_fine_tuning_IMDb.pth'
torch.save(net_trained.state_dict(), save_path)

In [57]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()
net_trained.to(device) 

epoch_corrects = 0

for batch in tqdm(test_dl):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    inputs = batch.Text[0].to(device) 
    labels = batch.Label.to(device)  

    with torch.set_grad_enabled(False):

        outputs = net_trained(inputs, token_type_ids=None, attention_mask=None,
                              output_all_encoded_layers=False, attention_show_flg=False)

        loss = criterion(outputs, labels)  
        _, preds = torch.max(outputs, 1)  
        epoch_corrects += torch.sum(preds == labels.data) 

epoch_acc = epoch_corrects.double() / len(test_dl.dataset)

print('테스트 데이터 {}개에서 정답률: {:.4f}'.format(len(test_dl.dataset), epoch_acc))

100%|██████████| 782/782 [08:14<00:00,  1.58it/s]

테스트 데이터 25000개에서 정답률: 0.8999


In [58]:
batch_size = 64
test_dl = torchtext.data.Iterator(
    test_ds, batch_size=batch_size, train=False, sort=False)

In [59]:
batch = next(iter(test_dl))

inputs = batch.Text[0].to(device)  
labels = batch.Label.to(device)  

outputs, attention_probs = net_trained(inputs, token_type_ids=None, attention_mask=None,
                                       output_all_encoded_layers=False, attention_show_flg=True)

_, preds = torch.max(outputs, 1) 

In [60]:
def highlight(word, attn):
    "Attention 값이 크면 문자 배경을 진한 빨간색으로 하는 html을 출력하는 함수"

    html_color = '#%02X%02X%02X' % (
        255, int(255*(1 - attn)), int(255*(1 - attn)))
    return '<span style="background-color: {}"> {}</span>'.format(html_color, word)


def mk_html(index, batch, preds, normlized_weights, TEXT):
    "HTML 데이터를 작성한다"

    sentence = batch.Text[0][index]  
    label = batch.Label[index]  
    pred = preds[index]  

    if label == 0:
        label_str = "Negative"
    else:
        label_str = "Positive"

    if pred == 0:
        pred_str = "Negative"
    else:
        pred_str = "Positive"

    html = '정답 라벨: {}<br>추론 라벨: {}<br><br>'.format(label_str, pred_str)

    for i in range(12):

        attens = normlized_weights[index, i, 0, :]
        attens /= attens.max()

        html += '[BERT의 Attention을 시각화_' + str(i+1) + ']<br>'
        for word, attn in zip(sentence, attens):

            if tokenizer_bert.convert_ids_to_tokens([word.numpy().tolist()])[0] == "[SEP]":
                break

            html += highlight(tokenizer_bert.convert_ids_to_tokens(
                [word.numpy().tolist()])[0], attn)
        html += "<br><br>"

    all_attens = attens*0
    for i in range(12):
        attens += normlized_weights[index, i, 0, :]
    attens /= attens.max()

    html += '[BERT의 Attention을 시각화_ALL]<br>'
    for word, attn in zip(sentence, attens):

        if tokenizer_bert.convert_ids_to_tokens([word.numpy().tolist()])[0] == "[SEP]":
            break

        html += highlight(tokenizer_bert.convert_ids_to_tokens(
            [word.numpy().tolist()])[0], attn)
    html += "<br><br>"

    return html

In [61]:
from IPython.display import HTML

index = 3 
html_output = mk_html(index, batch, preds, attention_probs, TEXT) 
HTML(html_output)


In [62]:
index = 61
html_output = mk_html(index, batch, preds, attention_probs, TEXT) 
HTML(html_output) 
